In [46]:
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [48]:
import pickle

In [49]:
result_path = "/home/kardosp/continuethelist/outputs/graphwalk_v2_output_sample4.pickle"

In [50]:
with open(result_path, "rb") as f:
    result_data = pickle.load(f)

In [71]:
from evaluation.evaluate_script import graph_eval_relaxed_ds
from helper.random_extra_functions import explanation_to_pd
graph_eval_relaxed_score, extra = graph_eval_relaxed_ds(result_data, extra_info=True)

100%|███████████████████████████████████████| 250/250 [00:00<00:00, 3920.47it/s]


In [72]:
q = 1
i = 0
row_counter = 0
for item, score in zip(result_data, extra):
    if score == 0:
        if i == q:
            # print(item["sparql_query"])
            # print("explanation:", item["explanation"])
            break
        else:
            i += 1
    row_counter += 1

In [73]:
row_counter

5

In [74]:
result_data[row_counter]["sparql_query"]

'SELECT DISTINCT ?uri WHERE {?uri <http://dbpedia.org/ontology/order> <http://dbpedia.org/resource/Bacillales> . ?uri <http://dbpedia.org/property/domain> <http://dbpedia.org/resource/Bacteria>  . ?uri <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Bacteria>}'

In [75]:
expl = result_data[row_counter]["explanation"]
df = explanation_to_pd(expl)

In [76]:
df

,start,relation,end,type
0,x,<http://dbpedia.org/ontology/domain>,<http://dbpedia.org/resource/Bacteria>,forward
1,x,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Bacteria>,forward
2,x,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Species>,forward
3,x,<http://dbpedia.org/property/domain>,<http://dbpedia.org/resource/Bacteria>,forward
4,x,<http://dbpedia.org/property/classis>,<http://dbpedia.org/resource/Bacilli>,forward
5,x,<http://dbpedia.org/ontology/class>,<http://dbpedia.org/resource/Bacilli>,forward


In [77]:
df[df["type"] == "forward"]

,start,relation,end,type
0,x,<http://dbpedia.org/ontology/domain>,<http://dbpedia.org/resource/Bacteria>,forward
1,x,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Bacteria>,forward
2,x,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Species>,forward
3,x,<http://dbpedia.org/property/domain>,<http://dbpedia.org/resource/Bacteria>,forward
4,x,<http://dbpedia.org/property/classis>,<http://dbpedia.org/resource/Bacilli>,forward
5,x,<http://dbpedia.org/ontology/class>,<http://dbpedia.org/resource/Bacilli>,forward


In [60]:
df[df["start"] == "<http://dbpedia.org/resource/Arkansas>"]

,start,relation,end


In [25]:
expl.keys()

dict_keys(['forward', 'invalids', 'backward', 'backward2'])

In [43]:
import pandas as pd
from evaluation.evaluate_script import normalize_triple

def explanation_to_pd(explanation):
    pd_rows = list()

    for k, v in explanation.items():
        if k == "invalids":
            continue
        if k == "backward2":
            for item in v:
                for triple in item[0]:
                    pd_rows.append(normalize_triple(triple))
        else:
            for triple in v:
                pd_rows.append(normalize_triple(triple))
    # return pd_rows
    return pd.DataFrame(pd_rows, columns=["start", "relation", "end"])

In [44]:
df = explanation_to_pd(expl)

In [45]:
df

,start,relation,end
0,x,<http://dbpedia.org/property/title>,<http://dbpedia.org/resource/List_of_U.S._stat...
1,x,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/AdministrativeReg...
2,x,<http://purl.org/linguistics/gold/hypernym>,<http://dbpedia.org/resource/State>
3,x,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/%3Chttp://purl.or...
4,x,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Location>
...,...,...,...
743,x,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://dbpedia.org/ontology/Agent>
744,x,<http://dbpedia.org/property/placeOfDeath>,x
745,x,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://xmlns.com/foaf/0.1/Person>
746,x,<http://dbpedia.org/property/placeOfDeath>,x


In [7]:
result_data[0]

{'_id': '2413',
 'corrected_question': 'Starwood operates in which places?',
 'intermediary_question': 'What is the <address> of the <hotels> whose <operator> is <Starwood Hotels and Resorts Worldwide>?',
 'sparql_query': 'SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/operator> <http://dbpedia.org/resource/Starwood_Hotels_and_Resorts_Worldwide> . ?x <http://dbpedia.org/property/address> ?uri  . }',
 'sparql_template_id': 5,
 'result': ['181',
  'http://dbpedia.org/resource/California',
  'http://dbpedia.org/resource/San_Francisco',
  '2',
  'Calle San Fernando, 2',
  '1',
  'http://dbpedia.org/resource/Chennai',
  'http://dbpedia.org/resource/Tamil_Nadu',
  'Krakowskie Przedmiescie 42/44',
  '1900',
  'http://dbpedia.org/resource/Seattle',
  '811',
  '333',
  '210',
  'http://dbpedia.org/resource/Atlanta',
  '10600',
  'http://dbpedia.org/resource/Westminster,_Colorado',
  'Main Gulshan Avenue, Plot-01, Road 45, Gulshan-2, Dhaka 1212, Bangladesh',
  '211',
  'http://dbpe

In [8]:
result_data[0]["explanation"]["backward2"]

[[[['?dy', 'http://dbpedia.org/property/address', '?uri'],
   ['?dy',
    'http://dbpedia.org/ontology/tenant',
    'http://dbpedia.org/resource/Starwood_Hotels_and_Resorts_Worldwide']],
  22],
 [[['?dr', 'http://dbpedia.org/property/address', '?uri'],
   ['?dr',
    'http://dbpedia.org/property/operator',
    'http://dbpedia.org/resource/Starwood_Hotels_and_Resorts_Worldwide']],
  24],
 [[['?dw', 'http://dbpedia.org/property/address', '?uri'],
   ['?dw',
    'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
    'http://dbpedia.org/ontology/Hotel']],
  628],
 [[['?ea', 'http://dbpedia.org/ontology/city', '?uri'],
   ['?ea',
    'http://purl.org/linguistics/gold/hypernym',
    'http://dbpedia.org/resource/Institution']],
  1610],
 [[['?ar', 'http://dbpedia.org/property/city', '?uri'],
   ['?ar',
    'http://purl.org/linguistics/gold/hypernym',
    'http://dbpedia.org/resource/Institution']],
  1617],
 [[['?io', 'http://dbpedia.org/ontology/deathPlace', '?uri'],
   ['?io',
    'http://w

In [62]:
from query.query_functions2 import run_sparql_query

In [63]:
run_sparql_query('SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Arkansas> <http://dbpedia.org/ontology/language> ?x . ?x <http://dbpedia.org/property/region> ?uri  . ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/EthnicGroup>}')

[{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/California'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Florida'}},
 {'uri': {'type': 'uri',
   'value': 'http://dbpedia.org/resource/Georgia_(U.S._state)'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Illinois'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Michigan'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/New_York'}},
 {'uri': {'type': 'uri',
   'value': 'http://dbpedia.org/resource/North_Carolina'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Ohio'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Pennsylvania'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Tennessee'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Texas'}}]

In [64]:
run_sparql_query('SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Arkansas> <http://dbpedia.org/ontology/language> ?x . ?x <http://dbpedia.org/property/region> ?uri}')

[{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/California'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Florida'}},
 {'uri': {'type': 'uri',
   'value': 'http://dbpedia.org/resource/Georgia_(U.S._state)'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Illinois'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Michigan'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/New_York'}},
 {'uri': {'type': 'uri',
   'value': 'http://dbpedia.org/resource/North_Carolina'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Ohio'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Pennsylvania'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Tennessee'}},
 {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Texas'}}]

In [82]:
#NEXT WORD:


variable_start = "aaa"
invalid_variable = ["uri"]

def next_word(word_state):
    i = -1
    new_word = ""
    done = False
    while not done:

        x = ord(word_state[i])
        x = x + 1
        if x == 123:
            new_word = "a"+new_word
            i-=1
        else:
            new_word = word_state[:i] + chr(x) + new_word
            done = True

    if new_word in invalid_variable:
        return next_word(new_word)
    return new_word

In [83]:
print(next_word("aaa"))
print(next_word("aaz"))
print(next_word("azz"))
print(next_word("czz"))

aab
aba
baa
daa


In [84]:

print(next_word("urh"))

urj
